In [5]:
#Install packages
! pip install beautifulsoup4
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

In [64]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import json 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt  
from matplotlib import style 
print("done")

done


In [7]:
#Search halifax communities in wikipedia, and extract them from Halifax Peninsula.
HalifaxUrl = 'https://en.wikipedia.org/wiki/Communities_in_the_Halifax_Regional_Municipality'
WholeHalifax = BeautifulSoup(requests.get(HalifaxUrl).text, 'lxml')
Halifax_Peninsula = WholeHalifax.find('table', class_="multicol").tbody.tr
HalifaxCommunities = []
for i in Halifax_Peninsula.find_all('li'):
    HalifaxCommunities.append(i.text)
for n, i in enumerate(HalifaxCommunities):
    if i == 'Albro Lake\nCrystal Heights\nHighfield Park':
        HalifaxCommunities[n] = 'Albro Lake'
    if i == 'Burnside':
        HalifaxCommunities[n] = 'Burnside, Halifax'
    if i == 'Cowie Hill':
        HalifaxCommunities[n] = 'Cowie Hill, Halifax'
    if i == 'Woodlawn':
        HalifaxCommunities[n] = 'Woodlawn, Halifax'
    if i == 'Manor Park':
        HalifaxCommunities[n] = 'Manor Park, Halifax'
    if i == 'Wellington':
        HalifaxCommunities[n] = 'Wellington, Halifax'
    if i == 'Waverley':
        HalifaxCommunities[n] = 'Waverley, Halifax'
    if i == 'Quinpool district':
        HalifaxCommunities[n] = 'Quinpool, Halifax'
    if i == 'Fairview':
        HalifaxCommunities[n] = 'Fairview, Halifax'
    if i == 'Fairmount':
        HalifaxCommunities[n] = 'Fairmount, Halifax'
    if i == 'Woodside':
        HalifaxCommunities[n] = 'Woodside, Halifax'
    if i == 'Richmond':
        del HalifaxCommunities[n]

In [ ]:
##  Extract latitudes and longitudes for each communities from Wikipedia.

#lat = []
#lon = []
#for i in HalifaxCommunities:
#    url = 'https://en.wikipedia.org/wiki/{}'.format(i)
#    r = requests.get(url).text
#    r = BeautifulSoup(r, 'lxml')
#    lat.append(r.find('span', class_="latitude").text)
#    lon.append(r.find('span', class_="longitude").text)
#cor = pd.DataFrame({'Communities':HalifaxCommunities, 'lat':lat, 'lon':lon})

##  Convert the coordinates into a right format and clean the names of communities

#def del_coor(x):
#    x = x.str.replace('.','').str.replace('″','').str.replace('′','').str.replace("°",".").str.replace('N','').str.replace('W','')
#    x = x.astype(float)
#    return x
    
#cor['lon'] = del_coor(cor['lon'])*-1
#cor['lat'] = del_coor(cor['lat'])
#cor['Communities'] = cor['Communities'].str.replace("Halifax","").str.replace(",","").str.replace("Nove Scotia","").str.strip()
#cor['Communities']
#cor

In [10]:
#The extracted coordinates from Wikipedia above is not up to date.
#Hence, append HalifaxCommunities list mannually and obtain communities coordinates from Geolocator.
lat = []
lon = []
for i in HalifaxCommunities:
    address = i + ', CA'
    geolocator = Nominatim(user_agent = "YHZ_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lat.append(latitude)
    lon.append(longitude)
    print(i,location.latitude,location.longitude)
YHZcor = pd.DataFrame({'communities':HalifaxCommunities, 'lat':lat, 'lon':lon})
YHZcor.head()

Downtown Halifax 44.6461752 -63.5725221
The Hydrostone 44.6641477 -63.6037432856765
North End Halifax 44.6550339 -63.5949305
West End, Halifax 44.6481851 -63.6172772
Quinpool, Halifax 44.64717195 -63.5965004821299
South End Halifax 44.6318333 -63.5800267
Spring Garden 44.64128565 -63.5811858125625
Armdale 44.6406015 -63.6224485
Boulderwood 44.6245031 -63.5982442
Clayton Park 44.6639666 -63.6525669
Cowie Hill, Halifax 44.6285559 -63.6215005
Fairview, Halifax 44.65515 -63.6384566
Fairmount, Halifax 44.6424475 -63.6324446
Jollimore 44.6299041 -63.6063922
Melville Cove 44.6330952 -63.6153365
Rockingham 44.6760232 -63.6616731
Spryfield 44.6174989 -63.6183631
Albro Lake 44.6853126 -63.5749927
Crystal Heights 44.6874216 -63.5833075
Highfield Park 44.6869672 -63.588839
Brightwood 44.6761567 -63.5772693
Burnside, Halifax 44.712138 -63.5902581
Crichton Park 44.6810198 -63.5657037
Downtown Dartmouth 44.6675083 -63.5686004
Imperoyal 44.6372717 -63.5253527
Manor Park, Halifax 44.6706939 -63.5459733

,communities,lat,lon
0,Downtown Halifax,44.646175,-63.572522
1,The Hydrostone,44.664148,-63.603743
2,North End Halifax,44.655034,-63.594930
3,"West End, Halifax",44.648185,-63.617277
4,"Quinpool, Halifax",44.647172,-63.596500


In [11]:
#Obtain the geograpical coordinate of Halifax
address = 'Halifax, CA'
geolocator = Nominatim(user_agent="YHZ_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
lat.append(latitude)
lon.append(longitude)
print('The geograpical coordinates of Halifax are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Halifax are 44.648618, -63.5859487.


In [12]:
#Maping Toronto's neighbourhood
YHZmap = folium.Map(location=[latitude, longitude], zoom_start=10, width = 800, height = 1000)

for lat, lng, borough in zip(YHZcor['lat'], YHZcor['lon'], YHZcor['communities']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(YHZmap)  
    
YHZmap

In [14]:
#Exploring the venues of Halifax
CLIENT_ID = '53IPUQGU1JWK24WFB1GCTYYXQ5MP4EQRCHRIDKTHLCSREIRP' 
CLIENT_SECRET = '4UCVZQWCTSAH3NW1XFKTI4MVEOLHJY1AT4LGDANRC2XP4LHN' 
VERSION = '20180605'
limit = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Halifax_venues = getNearbyVenues(names=YHZcor['communities'],
                                   latitudes=YHZcor['lat'],
                                   longitudes=YHZcor['lon']
                                  )

In [15]:
Halifax_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Halifax,44.646175,-63.572522,Stillwell Beer Bar,44.646908,-63.573875,Bar
1,Downtown Halifax,44.646175,-63.572522,Waterfront Boardwalk,44.647040,-63.570178,Waterfront
2,Downtown Halifax,44.646175,-63.572522,The Stubborn Goat Gastropub,44.645389,-63.574467,Gastropub
3,Downtown Halifax,44.646175,-63.572522,The Old Triangle Irish Alehouse,44.647654,-63.572307,Pub
4,Downtown Halifax,44.646175,-63.572522,Chives Canadian Bistro,44.645078,-63.572802,Restaurant
...,...,...,...,...,...,...,...
448,"Waverley, Halifax",44.782712,-63.598929,Wicky Clarke's Country Club,44.786018,-63.599890,Playground
449,"Waverley, Halifax",44.782712,-63.598929,Jigger's Rapid Response Rig,44.779924,-63.603322,Moving Target
450,"Wellington, Halifax",44.872754,-63.607648,Inner Strength Taekwondo,44.869598,-63.610799,Athletics & Sports
451,Windsor Junction,44.790638,-63.646073,Bobby Grace Furniture,44.791338,-63.643112,Furniture / Home Store


In [51]:
YHZ_onehot = pd.get_dummies(Halifax_venues[['Venue Category']], prefix="", prefix_sep="")
YHZ_onehot['Neighborhood'] = Halifax_venues['Neighbourhood']
fixed_columns = [YHZ_onehot.columns[-1]] + list(YHZ_onehot.columns[:-1])
YHZ_onehot = YHZ_onehot[fixed_columns]
YHZ_grouped = YHZ_onehot.groupby('Neighborhood').mean().reset_index()

# num_top_venues = 10

# for hood in YHZ_grouped['Neighborhood']:
#     print("----"+hood+"----")
#     temp = YHZ_grouped[YHZ_grouped['Neighborhood'] == hood].T.reset_index()
#     temp.columns = ['venue','freq']
#     temp = temp.iloc[1:]
#     temp['freq'] = temp['freq'].astype(float)
#     temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = YHZ_grouped['Neighborhood']

for ind in np.arange(YHZ_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(YHZ_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Albro Lake,Beach,Lake,Event Space,Food Stand,Food Court
1,Armdale,Electronics Store,Breakfast Spot,Hotel,Sandwich Place,Event Space
2,Bedford,Grocery Store,Coffee Shop,Pub,Sandwich Place,Breakfast Spot
3,Birch Cove,Food Stand,Food & Drink Shop,Garden Center,Gas Station,Chinese Restaurant
4,Boulderwood,Beach,Theater,Electronics Store,Food Court,Food & Drink Shop
5,Brightwood,Liquor Store,Sandwich Place,Grocery Store,Golf Course,Zoo
6,"Burnside, Halifax",Zoo,Wine Shop,Paper / Office Supplies Store,Furniture / Home Store,Electronics Store
7,Clayton Park,Grocery Store,Pharmacy,Coffee Shop,Lebanese Restaurant,Shipping Store
8,"Cowie Hill, Halifax",Pharmacy,Convenience Store,Gym,Coffee Shop,Dessert Shop
9,Crichton Park,Park,Garden Center,Zoo,Electronics Store,Food & Drink Shop


In [53]:
YHZ_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Beach,...,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Yoga Studio,Zoo
0,Albro Lake,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
1,Armdale,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
2,Bedford,0.040000,0.00,0.00,0.0,0.000000,0.000000,0.040000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
3,Birch Cove,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
4,Boulderwood,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
5,Brightwood,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
6,"Burnside, Halifax",0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.250000,0.000000,0.25
7,Clayton Park,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.040000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
8,"Cowie Hill, Halifax",0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
9,Crichton Park,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00


In [109]:
from sklearn.metrics import silhouette_score
score = []

for i in range(2,20):
    kclusters = i
    YHZ_grouped_clustering = YHZ_grouped.drop('Neighborhood', 1)
    kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0, n_init = 12).fit(YHZ_grouped_clustering)
    labels = kmeans.labels_
    score.append(silhouette_score(YHZ_grouped_clustering, labels, metric = 'euclidean'))
    print(i, silhouette_score(YHZ_grouped_clustering, labels, metric = 'euclidean'))

2 0.3876927615600245
3 0.2935812384304407
4 0.30373451097900633
5 0.22723643851363537
6 0.16714559532627654
7 0.2118693477626544
8 0.1792942664262567
9 0.2359487419944243
10 0.18864219849014485
11 0.16647737712777344
12 0.17901838716055074
13 0.14794967085118868
14 0.08584956786805416
15 0.1113863086126195
16 0.13608354372940967
17 0.13722839411394913
18 0.11628019779070779
19 0.12858348176672438


In [101]:
kclusters = 4
YHZ_grouped_clustering = YHZ_grouped.drop('Neighborhood', 1)
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0, n_init = 12).fit(YHZ_grouped_clustering)
labels = kmeans.labels_
labels

array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0,
       0, 1, 3, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [103]:
#neighborhoods_venues_sorted.drop('Cluster Labels',axis = 1, inplace = True)
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)

In [104]:
YHZ_merged = YHZcor
YHZ_merged = YHZ_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='communities')
YHZ_merged

,communities,lat,lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Halifax,44.646175,-63.572522,0.0,Coffee Shop,Restaurant,Pub,Hotel,Sushi Restaurant
1,The Hydrostone,44.664148,-63.603743,0.0,Rental Car Location,Café,Park,Caribbean Restaurant,Middle Eastern Restaurant
2,North End Halifax,44.655034,-63.594930,0.0,Brewery,Breakfast Spot,Coffee Shop,Pharmacy,Rock Club
3,"West End, Halifax",44.648185,-63.617277,0.0,Fast Food Restaurant,Coffee Shop,Clothing Store,Shoe Store,Pharmacy
4,"Quinpool, Halifax",44.647172,-63.596500,0.0,Pizza Place,Café,Fast Food Restaurant,Greek Restaurant,Yoga Studio
5,South End Halifax,44.631833,-63.580027,0.0,Coffee Shop,Moving Target,College Cafeteria,Zoo,Electronics Store
6,Spring Garden,44.641286,-63.581186,0.0,Coffee Shop,Breakfast Spot,Sushi Restaurant,Turkish Restaurant,Bar
7,Armdale,44.640602,-63.622448,0.0,Electronics Store,Breakfast Spot,Hotel,Sandwich Place,Event Space
8,Boulderwood,44.624503,-63.598244,0.0,Beach,Theater,Electronics Store,Food Court,Food & Drink Shop
9,Clayton Park,44.663967,-63.652567,0.0,Grocery Store,Pharmacy,Coffee Shop,Lebanese Restaurant,Shipping Store


In [105]:
YHZ_merged = YHZ_merged.dropna()
YHZ_merged['Cluster Labels'] = YHZ_merged['Cluster Labels'].astype(int)

/Users/zhenglouis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [106]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = ([i + x + (i*x)**2 for i in range(kclusters)])
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(YHZ_merged['lat'], YHZ_merged['lon'], YHZ_merged['communities'], YHZ_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1] ,
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [110]:
YHZ_merged.loc[YHZ_merged['Cluster Labels'] == 1, YHZ_merged.columns[[1] + list(range(4, YHZ_merged.shape[1]))]]

,lat,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
30,44.703605,Soccer Field,Food Truck,Food Stand,Food Court,Food & Drink Shop
44,44.801451,Soccer Field,Food Truck,Food Stand,Food Court,Food & Drink Shop


In [111]:
YHZ_merged.loc[YHZ_merged['Cluster Labels'] == 2, YHZ_merged.columns[[1] + list(range(4, YHZ_merged.shape[1]))]]

,lat,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
24,44.637272,Pizza Place,Zoo,Electronics Store,Food Court,Food & Drink Shop
42,44.738503,Pizza Place,Athletics & Sports,Zoo,Electronics Store,Food Court
48,44.872754,Athletics & Sports,Zoo,Event Space,Food Stand,Food Court


In [112]:
YHZ_merged.loc[YHZ_merged['Cluster Labels'] == 3, YHZ_merged.columns[[1] + list(range(4, YHZ_merged.shape[1]))]]

,lat,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,44.685313,Beach,Lake,Event Space,Food Stand,Food Court
26,44.673975,Beach,Bed & Breakfast,Food Stand,Food Court,Food & Drink Shop
29,44.740318,Lake,Zoo,Food Truck,Food Court,Food & Drink Shop
